In [132]:
import json
import numpy as np
import pandas as pd
import os
import subprocess


In [133]:
def find_best_odds(df):
    # Identify all the columns for team1 and team2 odds
    team1_cols = [col for col in df.columns if 'team1' in col]
    team2_cols = [col for col in df.columns if 'team2' in col]
    
    # Function to find the maximum odds and their source, ignoring None values
    def max_with_source(lst, source_lst):
        max_value = None
        max_source = None
        for i, value in enumerate(lst):
            if value is not None and (max_value is None or value > max_value):
                max_value = value
                max_source = source_lst[i]
        return max_value, max_source

    # Apply the function to each row to find the best odds for team1 and team2
    df['best_team1_odds'], df['best_team1_source'] = zip(*df.apply(
        lambda row: max_with_source([row[col] for col in team1_cols], team1_cols), axis=1))
    df['best_team2_odds'], df['best_team2_source'] = zip(*df.apply(
        lambda row: max_with_source([row[col] for col in team2_cols], team2_cols), axis=1))
    
    # Select only the relevant columns to display
    cols_to_display = ['match', 'best_team1_odds', 'best_team1_source', 'best_team2_odds', 'best_team2_source']
    return df[cols_to_display]

def arbitrage(row):
    implied_team1 = 1 / float(row['best_team1_odds'])
    implied_team2 = 1 / float(row['best_team2_odds'])
    if implied_team1 + implied_team2 < 1:
        return 1
    return 0

def open_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return np.array(data)

In [134]:


TAB_file = 'data/rugby_union/TAB.json'
sportsbet_file = 'data/rugby_union/sportsbet.json'

TAB_data = open_data(TAB_file)
sportsbet_data = open_data(sportsbet_file)

match_names = np.concatenate(([x['id'] for x in TAB_data], [x['id'] for x in sportsbet_data]))
match_names = np.unique(match_names)

df = pd.DataFrame({'match': match_names})

TAB_team1_dict = {obj['id']: obj['odds'][0] for obj in TAB_data}
TAB_team2_dict = {obj['id']: obj['odds'][2] for obj in TAB_data}
sportsbet_team1_dict = {obj['id']: obj['odds'][0] for obj in sportsbet_data}
sportsbet_team2_dict = {obj['id']: obj['odds'][2] for obj in sportsbet_data}


TAB_team1 = [TAB_team1_dict.get(match) for match in match_names]
TAB_team2 = [TAB_team2_dict.get(match) for match in match_names]
sportsbet_team1 = [sportsbet_team1_dict.get(match) for match in match_names]
sportsbet_team2 = [sportsbet_team2_dict.get(match) for match in match_names]

df['TAB_team1'] = TAB_team1
df['TAB_team2'] = TAB_team2
df['sportsbet_team1'] = sportsbet_team1
df['sportsbet_team2'] = sportsbet_team2



In [135]:
best_odds = find_best_odds(df).copy()
best_odds['arbitrage'] = best_odds.apply(lambda row: arbitrage(row), axis=1)
mask = best_odds['arbitrage'] == 1
best_odds[mask]


,match,best_team1_odds,best_team1_source,best_team2_odds,best_team2_source,arbitrage
16,Hurricanes Reds,1.30,TAB_team1,4.40,sportsbet_team2,1


In [136]:
def run_js_scripts(file_paths):
    processes = []
    for path in file_paths:
        # Form the command to execute 'node' with the file path
        command = ['node', path]
        
        # Spawn a new process for each command
        process = subprocess.Popen(command)
        
        # Append the process to the list of processes
        processes.append(process)

    # Wait for all processes to complete
    for process in processes:
        process.wait()
    print("All processes have completed.")
    return 0


def get_file_paths(directory_path):
    file_paths = []

    if not os.path.exists(directory_path):
        return file_paths
    
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

def clean_path(path):
    # List all items in the directory
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        # Check if it's a file or directory
        if os.path.isfile(item_path):
            os.remove(item_path)  # Remove the file

In [137]:

paths = get_file_paths('scripts/rugby_union')
run_js_scripts(paths)


{ team: 'Beziers', odds: '1.28' }
{ team: 'Draw', odds: '16.00' }
{ team: 'Brive', odds: '3.80' }
{ team: 'Highlanders', odds: '5.00' }
{ team: 'Draw', odds: '15.00' }
{ team: 'Blues', odds: '1.13' }
{ team: 'Rebels', odds: '1.40' }
{ team: 'Draw', odds: '12.50' }
{ team: 'Western Force', odds: '2.85' }
{ team: 'Aurillac', odds: '1.39' }
{ team: 'Draw', odds: '15.00' }
{ team: 'Nevers', odds: '3.15' }
{ team: 'Grenoble', odds: '1.06' }
{ team: 'Draw', odds: '26.00' }
{ team: 'Rouen', odds: '8.25' }
{ team: 'Mont-de-Marsan', odds: '1.06' }
{ team: 'Draw', odds: '26.00' }
{ team: 'Soyaux Angouleme', odds: '8.25' }
{ team: 'Montauban', odds: '1.46' }
{ team: 'Draw', odds: '15.00' }
{ team: 'Agen', odds: '2.85' }
{ team: 'Provence', odds: '1.19' }
{ team: 'Draw', odds: '18.00' }
{ team: 'Colomiers', odds: '4.80' }
{ team: 'Valence Romans Drome', odds: '1.22' }
{ team: 'Draw', odds: '17.00' }
{ team: 'US Dax', odds: '4.30' }
{ team: 'Edinburgh', odds: '1.18' }
{ team: 'Draw', odds: '18.00' 

0